In [ ]:
from google.colab import drive
drive.flush_and_unmount()

In [ ]:
# # Importing libraries
# from google.colab import drive
# import sys
# # Mount google drive
# drive.mount('/content/gdrive')
# # Changing path dirctory
# sys.path.append('/content/gdrive/My Drive/Colab Notebooks')


Mounted at /content/gdrive


In [ ]:
import os, sys

from google.colab import drive
drive.mount('/content/gdrive')

#create a path to save the module
nb_path = '/content/notebooks'
os.symlink('/content/gdrive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union

import pandas as pd
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
from google.colab import userdata
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
import pickle
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [ ]:
# pip install crewai_tools

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.4/463.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.3/342.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 8.2 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3959 sha256=cb3260be2c63de24ad6888890b2e5783602faca16d84b0adc880dfb59874a96c
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a1

In [ ]:
# pip -qqq install langchain-groq==0.1.5 --progress-bar off

In [ ]:
file_1='/content/sample_data/Loan.csv'

df = pd.read_csv(file_1)
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,120.0,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [ ]:
connection = sqlite3.connect("Loan.db")
df.to_sql(name="Loan", con=connection)

614

In [ ]:
@dataclass
class Event:
    event: str
    timestamp: str
    text: str


def _current_time() -> str:
    return datetime.now(timezone.utc).isoformat()


class LLMCallbackHandler(BaseCallbackHandler):
    def __init__(self, log_path: Path):
        self.log_path = log_path

    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        """Run when LLM starts running."""
        assert len(prompts) == 1
        event = Event(event="llm_start", timestamp=_current_time(), text=prompts[0])
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        """Run when LLM ends running."""
        generation = response.generations[-1][-1].message.content
        event = Event(event="llm_end", timestamp=_current_time(), text=generation)
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")

In [ ]:
llm = ChatGroq(
    temperature=0,
    groq_api_key='',
    # model_name="llama-3.1-70b-versatile"
    model="llama3-70b-8192"

)

In [ ]:

human = "{text}"
prompt = ChatPromptTemplate.from_messages([("human", human)])

chain = prompt | llm
response = chain.invoke(
    {
        "text": "Which company has better models OpenAI or Anthropic? Respond with just the company name."
    }
)

print(response.content)


Anthropic


In [ ]:
response.__dict__


{'content': 'Anthropic',
 'additional_kwargs': {},
 'response_metadata': {'token_usage': {'completion_tokens': 3,
   'prompt_tokens': 28,
   'total_tokens': 31,
   'completion_time': 0.009661848,
   'prompt_time': 0.000783303,
   'queue_time': 0.014549524000000001,
   'total_time': 0.010445151},
  'model_name': 'llama3-70b-8192',
  'system_fingerprint': 'fp_c1a4bcec29',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-c956847b-50dd-4434-b7b2-753dc11e8d50-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': None}

In [ ]:
db = SQLDatabase.from_uri("sqlite:///Loan.db")


In [ ]:
@tool("list_tables")
def list_tables() -> str:
    """List the available tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

In [ ]:
list_tables.run()


Using Tool: list_tables


'Loan'

In [ ]:
@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

In [ ]:
print(tables_schema.run("Loan"))


Using Tool: tables_schema

CREATE TABLE "Loan" (
	"index" INTEGER, 
	"Loan_ID" TEXT, 
	"Gender" TEXT, 
	"Married" TEXT, 
	"Dependents" TEXT, 
	"Education" TEXT, 
	"Self_Employed" TEXT, 
	"ApplicantIncome" INTEGER, 
	"CoapplicantIncome" REAL, 
	"LoanAmount" REAL, 
	"Loan_Amount_Term" REAL, 
	"Credit_History" REAL, 
	"Property_Area" TEXT, 
	"Loan_Status" TEXT
)

/*
3 rows from Loan table:
index	Loan_ID	Gender	Married	Dependents	Education	Self_Employed	ApplicantIncome	CoapplicantIncome	LoanAmount	Loan_Amount_Term	Credit_History	Property_Area	Loan_Status
0	LP001002	Male	No	0	Graduate	No	5849	0.0	120.0	360.0	1.0	Urban	Y
1	LP001003	Male	Yes	1	Graduate	No	4583	1508.0	128.0	360.0	1.0	Rural	N
2	LP001005	Male	Yes	0	Graduate	Yes	3000	0.0	66.0	360.0	1.0	Urban	Y
*/


In [ ]:
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

In [ ]:
execute_sql.run("SELECT * FROM Loan WHERE  ApplicantIncome	> 1000 LIMIT 5")


Using Tool: execute_sql


"[(0, 'LP001002', 'Male', 'No', '0', 'Graduate', 'No', 5849, 0.0, 120.0, 360.0, 1.0, 'Urban', 'Y'), (1, 'LP001003', 'Male', 'Yes', '1', 'Graduate', 'No', 4583, 1508.0, 128.0, 360.0, 1.0, 'Rural', 'N'), (2, 'LP001005', 'Male', 'Yes', '0', 'Graduate', 'Yes', 3000, 0.0, 66.0, 360.0, 1.0, 'Urban', 'Y'), (3, 'LP001006', 'Male', 'Yes', '0', 'Not Graduate', 'No', 2583, 2358.0, 120.0, 360.0, 1.0, 'Urban', 'Y'), (4, 'LP001008', 'Male', 'No', '0', 'Graduate', 'No', 6000, 0.0, 141.0, 360.0, 1.0, 'Urban', 'Y')]"

In [ ]:
@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Use this tool to double check if your query is correct before executing it. Always use this
    tool before executing a query with `execute_sql`.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})

In [ ]:
check_sql.run("SELECT * FROM Loan WHERE  ApplicantIncome	> 1000 LIMIT 5")


Using Tool: check_sql


'SELECT * FROM Loan WHERE ApplicantIncome > 1000 LIMIT 5'

In [ ]:

sql_dev = Agent(
    role="Senior Database Developer",
    goal="Construct and execute SQL queries based on a request",
    backstory=
        """
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `execute_sql` to check your queries for correctness.
        Use the `check_sql` to execute queries against the database.
    """
    ,
    llm=llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers



In [1]:
# pip -qqq install 'crewai[tools]'==0.32.0 --progress-bar off


In [ ]:
data_analyst = Agent(
    role="Senior Data Analyst",
    goal="You receive data from the database developer and analyze it",
    backstory=dedent(
        """
        You have deep experience with analyzing datasets using Python.
        Your work is always based on the provided data and is clear,
        easy-to-understand and to the point. You have attention
        to detail and always produce very detailed work (as long as you need).
    """
    ),
    llm=llm,
    allow_delegation=False,
)

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers



In [ ]:
report_writer = Agent(
    role="Senior Report Editor",
    goal="Write an executive summary type of report based on the work of the analyst",
    backstory=dedent(
        """
        Your writing still is well known for clear and effective communication.
        You always summarize long texts into bullet points that contain the most
        important details.
        """
    ),
    llm=llm,
    allow_delegation=False,
)

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers



In [ ]:
#Task
extract_data = Task(
    description="Extract data that is required for the query {query}.",
    expected_output="Database result for the query",
    agent=sql_dev,
)

In [ ]:
analyze_data = Task(
    description="Analyze the data from the database and write an analysis for {query}.",
    expected_output="Detailed analysis text",
    agent=data_analyst,
    context=[extract_data],
)

In [ ]:
write_report = Task(
    description=dedent(
        """
        Write an executive summary of the report from the analysis. The report
        must be less than 100 words.
    """
    ),
    expected_output="Markdown report",
    agent=report_writer,
    context=[analyze_data],
)


In [ ]:

crew = Crew(
    agents=[sql_dev, data_analyst, report_writer],
    tasks=[extract_data, analyze_data, write_report],
    process=Process.sequential,
    verbose=1,
    memory=False,
    output_log_file="crew.log",
)

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



In [ ]:

# inputs = {
#     "query": "Effects on Loan Status based on Gender and Applicant Income"
# }

result = crew.kickoff(inputs={"query": "Effects on Loan Status based on Gender and Applicant Income"})

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:LiteLLM call failed: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=llama3-70b-8192
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

# Agent: Senior Database Developer
## Task: Extract data that is required for the query Effects on Loan Status based on Gender and Applicant Income.


BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=llama3-70b-8192
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers